# 1. Function define

In [1]:
# 特定实体集中随机抽负例
import random
def generate_neg_sample(entity_list,gt):
	tmp_list = entity_list.copy()
	if gt in tmp_list: tmp_list.remove(gt)
	# np.random.shuffle(tmp_list)
	return random.sample(tmp_list,3)

def addtwodimdict(thedict, key_a, key_b, val): 
    if key_a in thedict:
        if key_b in thedict[key_a]:
            thedict[key_a][key_b].append(val)
        else:
            thedict[key_a].update({key_b: [val]})
    else:
        thedict.update({key_a:{key_b: [val]}})

def generate_bernoulli(p):
     rand_num = random.random()
     if rand_num < p:
          return 1
     else:
          return 0

def flat(a):
    flatted_a = []
    for _ in a:
        flatted_a += _
    return flatted_a

# 2. Collect KG's statistical information

In [2]:
import collections
f = open('./source/ecom_KG.jsonl')
relation_list = []

target_relation_dict = collections.defaultdict(set)
pa_relation_dict = {}
pa_relation_stat = {}
source_relation_neg = {}    # （源实体，关系）的负例收集
source_relation_pos = {}    # （源实体，关系）的正例收集

relation_cate = set()
# confidence = set()

for line in f.readlines():
    cur_dict = eval(line)
    relation_list.append(cur_dict["relation"])
    # confidence.add(cur_dict["status"])

    # 正例负例集合
    if cur_dict["status"] == '无关':
        addtwodimdict(source_relation_neg, cur_dict["source_name"], cur_dict["relation"], cur_dict["target_name"])
    elif cur_dict["status"] == "有关非常识" or cur_dict["status"] == "常识":
        addtwodimdict(source_relation_pos, cur_dict["source_name"], cur_dict["relation"], cur_dict["target_name"])

    target_relation_dict[cur_dict["relation"]].add(cur_dict["target_name"])
    pa_relation_stat[cur_dict["parent_relation"]] = pa_relation_stat.get(cur_dict["parent_relation"],0)+1
    if cur_dict["parent_relation"] not in pa_relation_dict: pa_relation_dict[cur_dict["parent_relation"]] = [cur_dict["relation"]]  
    else: 
        if cur_dict["relation"] not in pa_relation_dict[cur_dict["parent_relation"]]:
            pa_relation_dict[cur_dict["parent_relation"]].append(cur_dict["relation"])
    
    if cur_dict['related_item_category'] != '':
        relation_cate.add(cur_dict['relation'])
        

In [4]:
relation_list = list(set(relation_list))
entity_num = 0
for relation in relation_list:
    entity_num += len(target_relation_dict[relation])
print(entity_num)
print(pa_relation_stat)
print(pa_relation_dict)

f = open('./source/ecom_KG.jsonl','r')
num = len(f.readlines())

select_num = 10000
random.seed(5)

select_idx = random.sample(range(num),select_num)

309380
{'对应': 15613, '是一种': 2922124, '等义': 204042, '需要': 248430, '功能': 522028, '组合': 200, '适用': 556505, '口味': 200, '生产': 194052, '是一部分': 3238, '近义': 26192, '特产': 2596, '盛产': 10195, '气味': 200, '发生在': 423}
{'对应': ['人群对应动作', '动作对应人群', '口味对应品类', '款式对应品类', '气味对应品类', '气象状态对应动作'], '是一种': ['人群是一种人群', '动作是一种动作', '动物是一种动物', '口味是一种口味', '品类是一种品类', '工艺是一种工艺', '度量值是一种度量值', '时间是一种时间', '材质是一种材质', '款式是一种款式', '款式是一种风格', '气味是一种气味', '气象状态是一种气象状态', '空间是一种空间', '身体状况是一种身体状况', '身体部位是一种身体部位', '颜色是一种颜色', '风格是一种款式', '风格是一种风格'], '等义': ['人群等义人群', '功能等义功能', '动作等义动作', '动物等义动物', '口味等义口味', '品类等义品类', '工艺等义工艺', '度量值等义度量值', '时间等义时间', '材质等义材质', '款式等义款式', '款式等义风格', '气味等义气味', '气象状态等义气象状态', '空间等义空间', '身体状况等义身体状况', '身体部位等义身体部位', '颜色等义颜色', '风格等义款式', '风格等义风格'], '需要': ['人群需要功能', '人群需要品类', '人群需要款式', '功能需要品类', '功能需要款式', '动作需要功能', '动作需要品类', '时间需要功能', '时间需要品类', '时间需要材质', '时间需要款式', '气象状态需要功能', '气象状态需要品类', '气象状态需要材质', '空间需要功能'], '功能': ['功能', '功能是一种功能'], '组合': ['功能组合功能'], '适用': ['功能适用人群', '功能适用动作', '功能适用时间', '功能适用气象状态', '功能适用空间', '功能适用

# 3. Generate question prompt according to "realation" in KG triple

In [5]:
import numpy as np
f = open('./source/ecom_KG.jsonl','r')
system_prompt = "选择*选项*中相对来说最合适的一个填入空格处来补全以下*句子*，使其符合电子商务和生活消费领域中的逻辑和知识，\
输出选择的*选项*即可。括号内是词语的标签。\n*句子*：%s%s___（%s）%s\n*选项*：%s"
prompts = []

neg_set = []
pos_set = []
p = 0.4     # 下采样阈值
threshold = 10   # 负例candidate最少个数
# 随机种子
random.seed(2000)

ber_var_list = {}
for idx in select_idx:
    ber_var = generate_bernoulli(p)
    ber_var_list[idx] = ber_var

random.seed(2020)

for idx, line in enumerate(f.readlines()):
    if idx in select_idx:

        if cur_dict["parent_relation"] == '是一种' and ber_var_list[idx] == 0:
            continue
        question_dict = {}
        cur_dict = eval(line)
        ground_truth = cur_dict["target_name"]

        # 负例召回
        if cur_dict["source_name"] in source_relation_neg and cur_dict["relation"] in source_relation_neg[cur_dict["source_name"]]:
            neg_entity = source_relation_neg[cur_dict["source_name"]][cur_dict["relation"]]
        else:
            neg_entity = []   
        # 正例召回
        if cur_dict["source_name"] in source_relation_pos and cur_dict["relation"] in source_relation_pos[cur_dict["source_name"]]:
            pos_entity = source_relation_pos[cur_dict["source_name"]][cur_dict["relation"]]
        else:
            pos_entity = []
            
        # 选项candidate集合构造：负例-正例
        choice_candidate = list(set(neg_entity).difference(set(pos_entity)))

        if (cur_dict["status"] == "有关非常识" or cur_dict["status"] == "常识") and len(choice_candidate)>=threshold:

            neg_set.append(choice_candidate)
            pos_set.append(ground_truth)
            
            # 负例set中随机选取负例构成选项
            neg_sample = generate_neg_sample(choice_candidate,ground_truth) 
            multi_choice = neg_sample.copy()
            multi_choice.append(ground_truth)
            random.shuffle(multi_choice)

            # 后缀默认为空，需要时可以指定
            suffix = ''

            # 主语构造
            subject = cur_dict["source_name"]+'（'+cur_dict["source_class_re"]+'）'
            item = cur_dict["related_item_category"]
            if item != '':
                subject = subject+'的'+item
                # print(subject)
            
            # 谓语构造   
            relation_name = cur_dict["parent_relation"]     
            if relation_name == '等义' or relation_name == '近义':
                predicate = '等同于或高度相关于'
            elif relation_name == '是一种' or relation_name == '功能':
                predicate = '属于或类似于'
                if cur_dict["relation"] == '功能':
                    predicate = '可以或者应该'
            elif relation_name == '对应':
                predicate = '可以用于修饰或可以联想到'
            elif relation_name == '适用':
                predicate = '适用于'
                if cur_dict["relation"] == '款式适用动作' or cur_dict["relation"] == '适用动作':
                    suffix = '的时候'
                elif cur_dict["relation"] == '功能适用动作':
                    predicate = '对于'
                    suffix = '很重要'
            elif relation_name == '需要':
                predicate = '需要'
                if cur_dict["relation"] == '动作需要品类': predicate = '的时候需要'
                elif cur_dict["relation"] == '功能需要品类': 
                    predicate = '对于'
                    suffix = '很重要'
            else:
                predicate = cur_dict["parent_relation"]

            prompt = system_prompt%(subject,predicate,cur_dict["target_class_re"],suffix,multi_choice) 
            
            question_dict['idx'] = idx
            question_dict['question'] = prompt
            question_dict['ground_truth'] = ground_truth
            question_dict['relation'] = relation_name
            prompts.append(question_dict)

# 4. Save original questions

In [6]:
import csv

f = open('./question_generation/question_original.jsonl','w')
for prompt in prompts:
    f.write(str(prompt)+'\n')
f.close()

# 5. Negative sampling

## 5.1. (Stage 1) Sample Collecting

In [7]:
filename = open('./neg_sampling/choice_set.jsonl','w')
filename.write(str(neg_set)+'\n')
filename.write(str(pos_set))
filename.close()

neg_set_unique = list(set(flat(neg_set)))
pos_set_unique = list(set(pos_set))

# print(len(neg_set_unique))
# print(len(pos_set_unique))

header = ['id','text','category']
with open('./neg_sampling/entity_text.csv', 'w', encoding='UTF8') as f_csv:
    writer = csv.writer(f_csv)
    # write the header
    writer.writerow(header)
    cur_idx_base = 0
    for i,text in enumerate(neg_set_unique):
        data = [i,text,'neg']
        writer.writerow(data)
    cur_idx_base = i+1
    for i,text in enumerate(pos_set_unique):
        data = [cur_idx_base+i,text,'pos']
        writer.writerow(data)

### Infer and Load entity embedding

- use **bge-large** to infer text embeddings of all entities ("./neg_sampling/entity_text.csv") in the questions (negative and positive samples)

- save the embeddings in "./neg_sampling/entity_text_embedding.jsonl"

- load the embeddings to calculate semantic simlarity

In [8]:
f_emb = open('./neg_sampling/entity_text_embedding.jsonl','r')
embedding_dict = {}
for data in f_emb.readlines():
    cur_dict = eval(data)
    cur_text = cur_dict['text']
    cur_cate = cur_dict['category']
    cur_embedding = eval('['+cur_dict['vector']+']')
    embedding_dict[cur_text] = cur_embedding

## 5.2. (Stage 2) Coarse-grained Filtering

In [9]:
import numpy as np

def cos_sim(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num / denom

f = open('./neg_sampling/choice_set.jsonl','r')
l1, l2 = f.readlines()
neg_set = eval(l1)
# neg_set = flat(eval(l1))
pos_set = eval(l2)
threshold = 0.80
sample_num = 15


candidate_neg_set = []
for idx, (neg_data, pos_data) in enumerate(zip(neg_set,pos_set)):
    sim_list = []
    neg_list = []

    vec_pos = np.array(embedding_dict[pos_data])
    for cur_neg in neg_data:
        vec_neg = np.array(embedding_dict[cur_neg])
        cur_sim = cos_sim(vec_pos,vec_neg)
        if cur_sim < threshold:
            neg_list.append(cur_neg)
            sim_list.append(cur_sim)


    sorted_neg_data = sorted(neg_list, key=lambda x: sim_list[neg_list.index(x)])
    sorted_neg_data.reverse()

    candidate_neg = sorted_neg_data[:sample_num]

    # 选项不够则不过滤
    if len(sim_list)<5:
        candidate_neg = neg_data

    print('-'*100)
    print("ground_truth: ", pos_data)
    print("candidate: ", candidate_neg)
    print("cos_sim: ", sorted(sim_list,reverse=True)[:5])
    candidate_neg_set.append(candidate_neg)

----------------------------------------------------------------------------------------------------
ground_truth:  学习
candidate:  ['阅读', '旅游', '听歌', '旅行', '画画', '雕刻', '绘画', '跑步', '洗衣服', '做饭', '散步', '运动', '办公', '骑行', '出游']
cos_sim:  [0.5927718460721048, 0.5765014433636212, 0.5509710079474719, 0.5465437829921435, 0.5432156463944261]
----------------------------------------------------------------------------------------------------
ground_truth:  熬夜
candidate:  ['出差', '逛街', '度假', '化妆', '旅游', '约会', '租房', '旅行', '洗衣服', '拍照', '办公', '自拍', '娱乐', '面试']
cos_sim:  [0.5940964784874264, 0.5511584954956413, 0.5374676147186265, 0.5339965846499038, 0.5098452309516115]
----------------------------------------------------------------------------------------------------
ground_truth:  午睡
candidate:  ['裸睡', '夜行', '吃饭', '喝茶', '沐浴', '煮饭', '宴会', '散步', '旅游', '做饭', '聚会', '出差', '按摩', '旅行', '读书']
cos_sim:  [0.7239534934355861, 0.7168570631097072, 0.6535593328823985, 0.6483942326435748, 0.647429310947032]
------

## 5.3. (Stage 3) Fine-grained Filtering

In [14]:
system_prompt = '给定以下#问题#和它的#答案#，请从#候选#选项里按照#要求#选出三个负例用于构造一个有一定难度的多选题的错误选项，按照#输出格式#给出选择，然后给出理由。\n#要求#\n\
1. 负例作为错误选项虽然不适合用于补全#问题#中的#句子#，但是有不错的混淆效果，让人难以抉择\n2. 三个易负例之间有一定差异，能涵盖较多领域。\n#输出格式#\n负例：负例1，负例2，负例3\
\n#问题#\n选择一个合适的短语填入空格处来补全以下*句子*，使其符合电子商务领域中的逻辑和知识。括号内是词语的标签。\n*句子*：%s\n#答案#\n%s\n#候选#\n%s'

prompts = []
f = open('./question_generation/question_original.jsonl','r')
for idx, line in enumerate(f.readlines()):
    cur_dict = eval(line)
    question = cur_dict['question'].split('\n*选项*：')[0].split('\n*句子*：')[1]
    ground_truth = cur_dict['ground_truth']
    candidate = candidate_neg_set[idx]
    prompt = system_prompt%(question,ground_truth,candidate)
    prompts.append(prompt)
    print('-'*100)
    print(prompt)
f.close()

import csv

header = ['id', 'question']

with open('./LLM_filter/prompt.csv', 'w', encoding='UTF8') as f_csv:
    writer = csv.writer(f_csv)
    # write the header
    writer.writerow(header)
    for i,prompt in enumerate(prompts):
        data = [i+1,prompt]
        # write the data
        writer.writerow(data)

----------------------------------------------------------------------------------------------------
给定以下#问题#和它的#答案#，请从#候选#选项里按照#要求#选出三个负例用于构造一个有一定难度的多选题的错误选项，按照#输出格式#给出选择，然后给出理由。
#要求#
1. 负例作为错误选项虽然不适合用于补全#问题#中的#句子#，但是有不错的混淆效果，让人难以抉择
2. 三个易负例之间有一定差异，能涵盖较多领域。
#输出格式#
负例：负例1，负例2，负例3
#问题#
选择一个合适的短语填入空格处来补全以下*句子*，使其符合电子商务领域中的逻辑和知识。括号内是词语的标签。
*句子*：婴儿（人群）可以用于修饰或可以联想到___（动作）
#答案#
学习
#候选#
['阅读', '旅游', '听歌', '旅行', '画画', '雕刻', '绘画', '跑步', '洗衣服', '做饭', '散步', '运动', '办公', '骑行', '出游']
----------------------------------------------------------------------------------------------------
给定以下#问题#和它的#答案#，请从#候选#选项里按照#要求#选出三个负例用于构造一个有一定难度的多选题的错误选项，按照#输出格式#给出选择，然后给出理由。
#要求#
1. 负例作为错误选项虽然不适合用于补全#问题#中的#句子#，但是有不错的混淆效果，让人难以抉择
2. 三个易负例之间有一定差异，能涵盖较多领域。
#输出格式#
负例：负例1，负例2，负例3
#问题#
选择一个合适的短语填入空格处来补全以下*句子*，使其符合电子商务领域中的逻辑和知识。括号内是词语的标签。
*句子*：学生党（人群）可以用于修饰或可以联想到___（动作）
#答案#
熬夜
#候选#
['出差', '逛街', '度假', '化妆', '旅游', '约会', '租房', '旅行', '洗衣服', '拍照', '办公', '自拍', '娱乐', '面试']
-------------------------------------------------------

### Employ LLM (e.g., GPT4) as the filter to choose candidate for the questions

In [15]:
import pandas as pd
import collections 

df = pd.read_excel('./LLM_filter/result.xlsx')

prompt_list = df['提示词']
answer_list = df['回答']
success_list = df['是否成功']

negs_list = []

for idx,answer in enumerate(answer_list):
    if success_list[idx] != '失败':
        negs = {}
        neg, _ = answer.split('\n',1)
        neg = neg.replace('负例：','').split('，')
        negs['index'] = idx
        negs['neg'] = neg
        print(negs)
        negs_list.append(negs)

{'index': 0, 'neg': ['阅读', '听歌', '画画']}
{'index': 1, 'neg': ['逛街', '租房', '面试']}
{'index': 2, 'neg': ['喝茶', '旅游', '读书']}
{'index': 3, 'neg': ['经纪人', '公务员', '知识分子']}
{'index': 4, 'neg': ['女兵', '女史', '女主']}
{'index': 5, 'neg': ['亲戚', '父亲', '长辈']}
{'index': 6, 'neg': ['青少年学生', '高中学生', '大学新生']}
{'index': 7, 'neg': ['小青年', '初一孩子', '初三孩子']}
{'index': 8, 'neg': ['初中生', '中老年人', '孕妇']}
{'index': 9, 'neg': ['大学生', '上班族', '青年']}
{'index': 10, 'neg': ['大儿童', '中童', '中大儿童']}
{'index': 11, 'neg': ['中童', '成人', '医生']}
{'index': 12, 'neg': ['懒汉', '穷光蛋', '果粉']}
{'index': 13, 'neg': ['壮年人', '宝妈', '幼童']}
{'index': 14, 'neg': ['胖妹', '大娘', '美娃']}
{'index': 15, 'neg': ['奶爸', '狗妈', '潮宝宝']}
{'index': 16, 'neg': ['同伙', '邻里', '对手']}
{'index': 17, 'neg': ['女编', '组员', '暖妈']}
{'index': 18, 'neg': ['软妹子', '胖宝宝', '高祖母']}
{'index': 19, 'neg': ['学龄儿童', '中大小童', '婴幼儿']}
{'index': 20, 'neg': ['修身显瘦', '晾晒', '嫩肤']}
{'index': 21, 'neg': ['养生', '助眠', '节能']}
{'index': 22, 'neg': ['化妆品架', '自行车货架', '浴室三角架']}
{'index': 23, 'neg': [

/Users/liulangming/miniforge3/envs/py310/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 6. Save refined questions

In [17]:
import random
import json

querys = []
f = open('./question_generation/question_original.jsonl','r')
f_out =open('./question_generation/question_refined.jsonl','w')
for idx, line in enumerate(f.readlines()):
    cur_dict = eval(line)
    question = cur_dict['question'].split('\n*选项*：')[0]
    ground_truth = cur_dict['ground_truth']

    # update candidate choice
    negs = negs_list[idx]
    candidate = negs['neg']+[ground_truth]
    # shuffle choice
    random.shuffle(candidate)
    # concate question and candidate choice
    query = question+'\n*选项*：'+str(candidate)
    # question knowledge dimension
    if cur_dict['relation'] == '是一种':
        dim = 'dim_2'
    else:
        dim = 'dim_1'
        
    info = {"id":idx+1, "question": query, "gt": ground_truth, "dim": dim}
    json.dump(info, f_out, ensure_ascii=False)
    f_out.write('\n')
f.close()
f_out.close()